In [2]:
import os
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
prototxt_path = os.path.join('model_data/deploy.prototxt')
caffemodel_path = os.path.join('model_data/weights.caffemodel')

# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)
print("Model_loaded")

Model_loaded


In [8]:
# load json and create model
#load the model 
from keras.models import model_from_json
json_file = open('FaceVerification.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("FaceVerification.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
# Compiling and Making New Predictions

# compile loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
MODE = "MPI"
if MODE is "COCO":
    prototxt_path = os.path.join('models/pose_deploy_linevec.prototxt')
    caffemodel_path = os.path.join('models/pose_iter_440000.caffemodel')
    nPoints = 18
    POSE_PAIRS = [ [1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]


elif MODE is "MPI" :
    prototxt_path = os.path.join('models/pose_deploy_linevec_faster_4_stages.prototxt')
    caffemodel_path = os.path.join('models/pose_iter_160000.caffemodel')
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(5, 30)
frameRate = cap.get(5)
print(frameRate)
count=0
while True:
    count+=1
    frameId=count
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=1000)
    (h, w) = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # construct a blob from the image
    #imageBlob = cv2.dnn.blobFromImage(
        #cv2.resize(frame, (300, 300)), 1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    imageBlob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104, 117, 123], False, False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    model.setInput(imageBlob)
    detections = model.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
    # extract the confidence (i.e., probability) associated with
    # the prediction
        confidence = detections[0, 0, i, 2]
        frameHeight= frame.shape[2]
        # filter out weak detections
        if confidence > 0.5:
        # compute the (x, y)-coordinates of the bounding box for
        # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), int(round(frameHeight/150)), 8)
            #Calling the model
            filename ="vijay.jpg"
            crop_img = frame[startY:endY, startX:endX]
            cv2.imwrite(filename, crop_img) 
            test_image = image.load_img("vijay.jpg", target_size = (64, 64))
            test_image = image.img_to_array(test_image)
            test_image = np.expand_dims(test_image, axis = 0)
            result = loaded_model.predict(test_image)  
            #training_set.class_indices
            if result[0][0] == 1:
                prediction = 'Found Vijay'
            elif result[0][0] == 0 :
                prediction = 'NotVijay--Someone'
            print(prediction)
            cv2.putText(frame, prediction, (startX, startY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
        cv2.imshow("Frame", frame)
    k= cv2.waitKey(5) & 0xff
    if k==27:
        break
print("vijay")
cv2.destroyAllWindows()
cap.release()